In [ ]:
import os, sys
from SPACEL.setting import set_environ_seed

set_environ_seed(42)
from SPACEL import Spoint
import scanpy as sc
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib
import seaborn as sns

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.serif"] = ["Arial"]
sc.settings.set_figure_params(
    dpi=50, dpi_save=300, facecolor="white", fontsize=10, vector_friendly=True, figsize=(3, 3)
)
sc.settings.verbosity = 3

In [ ]:
sample = "SS200000118TL_C1"

In [ ]:
od = f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/cell_spoint/{sample}/"
os.system(f"mkdir -p {od}")
os.chdir(od)

In [ ]:
import yaml

with open("/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/sample.yaml", "r") as file:
    sample_dict = yaml.safe_load(file)
samplelist = list(sample_dict.keys())
with open("/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/patient.yaml", "r") as file:
    patient_dict = yaml.safe_load(file)

In [ ]:
sc_ad_total = sc.read(
    f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/yixianai/SC_reanalysis/result/5_normal_diff/seurat_merge_rename.h5ad"
)
sc_ad_total2 = sc_ad_total.raw.to_adata()
sc_ad_total = sc_ad_total2
patient = patient_dict[sample]
if patient in sc_ad_total.obs["Patients"].values:
    sc_ad = sc_ad_total[
        ~((sc_ad_total.obs["Patients"] != patient) & (sc_ad_total.obs["cellTypes_filt"] == "TumorCell"))
    ]
else:
    sc_ad = sc_ad_total
st_ad_total = sc.read(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v2/adata_cellbin_total.h5ad"
)
st_ad = st_ad_total[st_ad_total.obs["batch"] == sample]

sc.pp.filter_genes(st_ad, min_cells=1)
sc.pp.filter_genes(sc_ad, min_cells=1)
sc.pp.filter_cells(st_ad, min_genes=1)
sc.pp.filter_cells(sc_ad, min_genes=1)

In [ ]:
spoint = Spoint.init_model(sc_ad, st_ad, celltype_key="cellTypes_nmf", sm_size=500000, n_threads=30)

In [ ]:
spoint.train(max_steps=5000, batch_size=4096)

In [ ]:
pre = spoint.deconv_spatial()

In [ ]:
st_ad = spoint.st_ad

In [ ]:
st_ad.write(f"Spiont_{sample}.h5ad")

In [ ]:
sc.pl.spatial(st_ad, color=pre.columns, ncols=6, vmax="p99", spot_size=100, save=f"embeddings_{sample}.pdf")